# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()

# Choosing a name for experiment
experiment_name = 'AutoML-Experiment'

experiment=Experiment(ws, experiment_name)
#project_folder = './pipeline-project'


# Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


# Dataset


TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


The dataset "Heart Failure Prediction" contains 12 clinical features of 299 patients with heart failure and a target variable "DEATH EVENT" indicating if the patient deceased during the follow-up period (boolean). Machine Learning can help detect and manage high-risk patients at an earlier stage. 

To solve this binary classification problem, I will use the 12 features to predict possible death events with the help of an ML algorithm.  


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
found = False
key = "Heart Failure Prediction"
description_text = "Heart Failure Prediction DataSet for Udacity Project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("The dataset is loaded")

if not found:
        # Creating Dataset and register it into Workspace
        example_data = "https://raw.githubusercontent.com/EugeniaSilantjeva/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Registering Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

The dataset is loaded


In [5]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [6]:
df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [7]:
df[df.isnull().any(axis=1)]

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT


In [8]:
df["DEATH_EVENT"].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [10]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
# Retrieving and saving the best automl model
automl_best_run, fitted_automl_model = automl_run.get_output() 

run_dict = automl_best_run.get_details()

print("Best Run ID: ", run_dict["runId"])
print("---"*10)
print("AutoML best run AUC_weighted: ", automl_best_run.get_metrics(name="AUC_weighted"))
print("---"*10)
print("Metrics: ", automl_best_run.get_metrics())
print("---"*10)
print("Fitted model and its hyperparameters : ", fitted_automl_model.steps[-1])



Best Run ID:  AutoML_7d1e2152-412f-4c53-ac0c-3f4f05d0ed3f_41
------------------------------
AutoML best run AUC_weighted:  {'AUC_weighted': 0.9218579454785403}
------------------------------
Metrics:  {'weighted_accuracy': 0.8758864499109998, 'matthews_correlation': 0.6585505541411006, 'recall_score_micro': 0.849425287356322, 'f1_score_weighted': 0.84358590455737, 'norm_macro_recall': 0.61791566779267, 'AUC_micro': 0.9222750693618709, 'recall_score_macro': 0.8089578338963351, 'precision_score_weighted': 0.8612728428742672, 'average_precision_score_micro': 0.9274960049064708, 'AUC_macro': 0.9218579454785403, 'log_loss': 0.41951009244079207, 'precision_score_macro': 0.8527702382158905, 'precision_score_micro': 0.849425287356322, 'f1_score_macro': 0.817037570981715, 'recall_score_weighted': 0.849425287356322, 'average_precision_score_weighted': 0.9338077377500668, 'AUC_weighted': 0.9218579454785403, 'accuracy': 0.849425287356322, 'average_precision_score_macro': 0.9108967096230177, 'f1_sc

In [13]:
print("AutoML Run Summary: ", automl_run.summary())

AutoML Run Summary:  [['StackEnsemble', 1, 0.9090671778259878], ['VotingEnsemble', 1, 0.9205699647945071], ['Failed', 2, nan], ['RandomForest', 16, 0.9154799974243909], ['LightGBM', 4, nan], ['XGBoostClassifier', 4, 0.9089099837731132], ['ExtremeRandomTrees', 9, 0.8976559833653083], ['KNN', 2, 0.7496161006793443], ['SGD', 1, 0.870688942657478]]


In [13]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model', model_path = './outputs/model.pkl')

In [15]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136370', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-136370'), name=automl_best_model, id=automl_best_model:1, version=1, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
automl_best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_d0cb29efe9761e5ec53d53c7a02528adf3bac468acfc52112c8981ab02a2dd18_d.txt',
 'azureml-logs/65_job_prep-tvmps_d0cb29efe9761e5ec53d53c7a02528adf3bac468acfc52112c8981ab02a2dd18_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_d0cb29efe9761e5ec53d53c7a02528adf3bac468acfc52112c8981ab02a2dd18_d.txt',
 'confusion_matrix',
 'logs/azureml/91_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/dataprep/python_span_a672ee0e-a09a-48cd-8451-4fcf1a269df3.jsonl',
 'logs/azureml/dataprep/python_span_ad15d4ec-ed0b-48d5-8768-a289ab3f72c3.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [17]:
# Reference: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

# Downloading the scoring file 

automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py' , 'score.py')


In [38]:
# Downloading conda environment 
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml' , 'env.yml')

In [18]:
from azureml.core import Model
# Creating the environment
#env=Environment.from_conda_specification(name='env',file_path = "env.yml")
env = Environment.get(workspace=ws, name="AzureML-AutoML")
# Creating an inference config
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

# Deploying the model as a web service to an Azure Container Instance (ACI)
service_name = 'hfprediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

webservice.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
print(webservice.state)

Healthy


In [20]:
print(webservice.get_logs())

/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-28T10:44:11,792105074+00:00 - rsyslog/run 
2021-01-28T10:44:11,792106074+00:00 - gunicorn/run 
2021-01-28T10:44:11,794170252+00:00 - iot-server/run 
2021-01-28T10:44:11,797356974+00:00 - nginx/run 
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:
# Reference: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb 
# and endpoint.py file used in Project 2
scoring_uri = "http://c27402a6-6dd0-4ce8-a9f4-ba6da94647c6.southcentralus.azurecontainer.io/score"
########## OR
df = df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
    'data': df[0:2].to_dict(orient='records')
})


print(input_data)
    

{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 7861, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 6}]}


In [30]:
with open("data.json", "w") as _f:
    _f.write(input_data)
    
#######################################################    

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
  

In [34]:
print(resp.json()) 

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
# Retrieve logs for this Webservice
webservice.get_logs()


'/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-28T10:44:11,792105074+00:00 - rsyslog/run \n2021-01-28T10:44:11,792106074+00:00 - gunicorn/run \n2021-01-28T10:44:11,794170252+00:00 - iot-server/run \n2021-01-28T10:44:11,797356974+00:00 - nginx/run 

In [36]:
# Delete service
webservice.delete()